## Crypto currency strategy testing

#### Load Essentials
In this notebook we will be using the [python-binance](https://python-binance.readthedocs.io/en/latest/) module combined with [backtesting](https://pypi.org/project/Backtesting/) module to create an enviroment that will allow us to backtest strategies on cryptocurrency. Given that you are in the US, this will require an api from [Binance-US](https://www.binance.us/en/home) and when you load the python-binance module, you will need to make changes listed [here](https://www.reddit.com/r/BinanceExchange/comments/dahxcq/binance_us_api_python_wrapper/). 

In [17]:
# Include a step to read in api key/secret from a separate file
import confidential
api_key = confidential.api_key
api_secret = confidential.api_secret

# Import modules
import tools
import pandas as pd
import volatility_inidicators
import volume_indicators

#### Create client using the python-binance module

In [18]:
# Import Binance module to create client
from binance.client import Client
client = Client(api_key, api_secret)

#### Get coin data using the client

In [19]:
timestamp = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
print(pd.to_datetime(timestamp, unit='ms'))

2019-09-23 00:00:00


#### Data Pre-processing
Using the client and get_symbol_ticker method we will get the price and clean it so that it will fit when we backtest. 

In [73]:
# get latest price from Binance API
btc_price = client.get_symbol_ticker(symbol="BTCUSDT")

# request historical candle (or klines) data
bars = client.get_historical_klines('BTCUSDT', '1d', timestamp, limit=2000)

for line in bars:
    del line[11:]
    
# create a Pandas DataFrame and export to CSV
import pandas as pd
btc_df = pd.DataFrame(bars, columns=['date', 'Open', 'High', 'Low', 'Close',
                                     'Volume','close_time','quote_asst_vol',
                                     'num_trades','taker_base_asst_vol','taker_quote_asst_vol'])
btc_df.set_index('date', inplace=True)
btc_df.index = pd.to_datetime(btc_df.index, unit='ms')

# using dictionary to convert specific columns
convert_dict = {'Open': float,
                'High': float,
                'Low': float,
                'Close': float,
                'Volume': float,
                'quote_asst_vol': float,
                'num_trades': int,
                'taker_base_asst_vol': float,
                'taker_quote_asst_vol': float
               }
btc_df = btc_df.astype(convert_dict)
btc_df.close_time = pd.to_datetime(btc_df.close_time, unit='ms')

btc_df.head()

,Open,High,Low,Close,Volume,close_time,quote_asst_vol,num_trades,taker_base_asst_vol,taker_quote_asst_vol
date,,,,,,,,,,
2019-09-23,9930.13,9930.13,9930.13,9930.13,0.001000,2019-09-23 23:59:59.999,9.930130e+00,1,0.000000,0.000000e+00
2019-09-24,9637.93,9665.05,7996.45,8519.67,202.350799,2019-09-24 23:59:59.999,1.739432e+06,2345,125.935464,1.075624e+06
2019-09-25,8528.12,8724.77,8247.78,8435.28,55.417971,2019-09-25 23:59:59.999,4.693617e+05,763,29.059427,2.460492e+05
2019-09-26,8434.45,8453.20,7761.87,8064.41,41.880079,2019-09-26 23:59:59.999,3.409237e+05,1023,19.808849,1.611154e+05
2019-09-27,8078.75,8259.96,7858.58,8178.73,47.142904,2019-09-27 23:59:59.999,3.799180e+05,990,23.903742,1.927560e+05


#### Create a backtesting environment for Coin Strategy

Here we create a simple strategy using simple moving average crossover. Compared to the cross function built in the [backtesting.lib](https://kernc.github.io/backtesting.py/doc/backtesting/lib.html#gsc.tab=0), which indicates when either SMA(n) where n is the window of the moving average crosses regardless of direction. The crossover function returns true when SMA(n1) crosses over SMA(n2). 

In [77]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA

class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

In [78]:
btc_df[['Open', 'High', 'Low', 'Close','Volume']].dtypes

Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
dtype: object

#### Visualizing backtest results

The following will allow us to see when the backtest trades and how much it will end up will. There is a detailed output that can be obtained as well. Looking at the results the [sharpe ratio](https://www.investopedia.com/terms/s/sharperatio.asp) indicates that the model is sub-optimal, but is expected since the model is simple and not optimized. This could be possibly improved by optimizing the parameters or adding additional data. 

In [82]:
bt = Backtest(btc_df[['Open', 'High', 'Low', 'Close','Volume']], SmaCross,
              cash=10000, commission=.001,
              exclusive_orders=True)

output = bt.run()
bt.plot()

<ipython-input-82-8f5f2ae97cfa>:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(btc_df[['Open', 'High', 'Low', 'Close','Volume']], SmaCross,


In [83]:
output

Start                     2019-09-23 00:00:00
End                       2021-04-19 00:00:00
Duration                    574 days 00:00:00
Exposure Time [%]                     76.1739
Equity Final [$]                      30043.7
Equity Peak [$]                       45860.8
Return [%]                            200.437
Buy & Hold Return [%]                 473.431
Return (Ann.) [%]                     101.034
Volatility (Ann.) [%]                 103.828
Sharpe Ratio                          0.97309
Sortino Ratio                         3.41581
Calmar Ratio                          2.80564
Max. Drawdown [%]                     -36.011
Avg. Drawdown [%]                    -6.07496
Max. Drawdown Duration      101 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                   20
Win Rate [%]                               40
Best Trade [%]                        182.208
Worst Trade [%]                      -19.7556
Avg. Trade [%]                    

#### Follow-up with

1. Gather additional data for model 
2. Improve and optimize the model
3. Execute model python-binance client